# 4.5 학습 알고리즘 구현하기
### 신경망 학습의 순서
0. **전제**
- 신경망에서 적응 가능한 가중치와 편향이 존재 <br/>
- 학습 : 이런 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정
1. **미니배치**
- 훈련 데이터 중 일부 가져옴(미니배치) -> 이것의 손실함수 값 줄이는 게 목표
2. **기울기 산출**
- 미니배치의 손실 함수 값 줄이기 위해 각 가중치의 매개변수의 기울기 구함
- 기울기는 손실 함수의 값을 가장 작게 하는 방향 제시
3. **매개변수 갱신**
- 가중치 매개변수를 기울기의 방향으로 아주 조금 갱신
4. **반복**
- 1~3단계 반복

### [1] 2층 신경망 클래스 구현 ###

In [1]:
# 확률적 경사 하강법(SGD) : 무작위로 고른 데이터에 대한 경사 하강법
# 2층 신경망(은닉층 1개)를 대상으로 손글씨 숫자 학습 신경망 구현
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
  # 초기화 수행 함수 (입력층의 뉴런 수, 은닉층의 뉴런 수, 출력층의 뉴런 수)
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # 가중치 초기화
    self.params = {}
    # 1층의 가중치
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    # 1층의 편향
    self.params['b1'] = np.zeros(hidden_size)
    # 2층의 가중치
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    # 2층의 편향
    self.params['b2'] = np.zeros(output_size)
  
  # 예측(추론) 수행 함수 (x는 입력 데이터)
  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y
  
  # 손실 함수 값 구하는 함수 (x : 입력 데이터, t : 정답 레이블)
  def loss(self, x, t):
    y = self.predict(x)

    return cross_entropy_error(y, t)

  # 정확도 구하는 함수
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy
  
  # 가중치 매개변수의 기울기 구하는 함수 (x : 입력 데이터, t : 정답 레이블)
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

### [2] 미니배치 학습 구현 ###

In [3]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
  load_mnist(normalize=True, one_hot_label=True)

train_lost_list = []

# 하이퍼파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)
  # grad = network.gradient(x_batch, t_batch) -> 성능 개선판

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2') :
    network.params[key] -= learning_rate * grad[key]
  
  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_lost_list.append(loss)

  # 학습 횟수가 늘어날수록 손실 함수의 값 줄어듦


KeyboardInterrupt: 

### [3] 시험 데이터로 평가하기 ###

In [4]:
# 오버피팅 되진 않았는지 확인해야 함
# 에폭(epoch) : 1에폭 = 학습에서 훈련 데이터 모두 소진했을 때 횟수
# ex) 훈련 데이터 1만개를 100개의 미니배치로 학습시 확률적 경사 하강법 100회 반복하면 훈련 데이터 모두 소진하기 때문에 이때는 100회가 1에폭
# (위의 함수에서 ### 부분만 추가하여 수정)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
  load_mnist(normalize=True, one_hot_label=True)

train_lost_list = []
train_acc_list = []   ###
test_acc_list = []    ###

# 하이퍼파라미터
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

### 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)
  # grad = network.gradient(x_batch, t_batch) -> 성능 개선판

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2') :
    network.params[key] -= learning_rate * grad[key]
  
  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_lost_list.append(loss)

  ### 1에폭당 정확도 계산
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.0993, 0.1032


KeyboardInterrupt: 